In [68]:
import pandas as pd
import pickle
with open('Tweets/tweets_adr.p', 'rb') as f:
    u = pickle._Unpickler(f)
    u.encoding = 'latin1'
    data = u.load()

In [69]:
strings = [" ".join(x['tokens']) for x in data[0]]
target = [x['y'] for x in data[0]]

In [70]:
from sklearn.model_selection import train_test_split
train_idx, test_idx = train_test_split(range(len(strings)), test_size = 0.2, stratify=target, random_state=13020)

In [71]:
len(train_idx), len(test_idx)

(17003, 4251)

In [72]:
X_train = [strings[i] for i in train_idx]
X_test = [strings[i] for i in test_idx]

y_train = [target[i] for i in train_idx]
y_test = [target[i] for i in test_idx]

In [73]:
import vectorizer
vec = vectorizer.Vectorizer(min_df=2)

In [74]:
vec.fit(X_train + X_test)

In [75]:
vec.seq_text = {}
vec.seq_text['train'] = vec.get_seq_for_docs(X_train)
vec.seq_text['test'] = vec.get_seq_for_docs(X_test)

In [76]:
vec.label = { 'train' : y_train, 'test' : y_test } 

In [77]:
from torchtext.vocab import Vectors, GloVe, CharNGram, FastText
url = 'https://s3-us-west-1.amazonaws.com/fasttext-vectors/wiki.simple.vec'
vectors = Vectors('wiki.simple.vec', url=url)

In [78]:
vec.word_dim = vectors.dim

In [79]:
import numpy as np
vec.embeddings = np.zeros((len(vec.idx2word), vec.word_dim))
for i, word in vec.idx2word.items() :
    vec.embeddings[i] = vectors[word].numpy()

In [80]:
import pickle
pickle.dump(vec, open('Tweets/vec_adr.p', 'wb'))